In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
from peft import PeftModel
import torch
import json


/home/user12/.conda/envs/pasha/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
sample = json.loads(open('/home/user12/prompts/valid.jsonl').readline())          # одна строка
messages = sample["messages"]                                # list[dict]

# Обрезаем всё после последнего user
messages_no_answer = [
    m for m in messages if m["role"] != "assistant"
]

prompt = tokenizer.apply_chat_template(
    messages_no_answer,
    tokenize=False,
    add_generation_prompt=True        # «<|im_start|>assistant\n» в конце
)

In [55]:
messages_no_answer.append({'role': 'assistant', 'content': ''})
messages_no_answer

[{'role': 'system',
  'content': 'You are a chemist. For each atom pair within 2 Å classify the bond type. Labels: 0 no-bond, 1 single, 2 double, 3 triple, 4 aromatic. Return ONLY JSON list [{"pair":[i,j],"label":n}].'},
 {'role': 'user',
  'content': 'Pairs within 2 Å:\n[0,0]=<p077>\n[0,1]=<p121>\n[1,0]=<p046>\n[1,1]=<p117>\n[1,2]=<p115>\n[2,1]=<p115>\n[2,2]=<p060>\n[2,3]=<p113>\n[3,2]=<p115>\n[3,3]=<p025>\n[3,4]=<p124>\n[3,5]=<p083>\n[4,3]=<p010>\n[4,4]=<p117>\n[5,3]=<p083>\n[5,5]=<p025>\n[5,6]=<p107>\n[5,27]=<p065>\n[6,5]=<p028>\n[6,6]=<p117>\n[6,7]=<p115>\n[7,6]=<p115>\n[7,7]=<p060>\n[7,8]=<p115>\n[8,7]=<p115>\n[8,8]=<p117>\n[8,9]=<p016>\n[8,26]=<p086>\n[9,8]=<p038>\n[9,9]=<p034>\n[9,10]=<p017>\n[9,11]=<p096>\n[10,9]=<p016>\n[10,10]=<p117>\n[11,9]=<p032>\n[11,11]=<p025>\n[11,12]=<p120>\n[12,11]=<p120>\n[12,12]=<p060>\n[12,13]=<p016>\n[12,25]=<p104>\n[13,12]=<p038>\n[13,13]=<p034>\n[13,14]=<p075>\n[13,15]=<p096>\n[13,16]=<p019>\n[14,13]=<p096>\n[14,14]=<p025>\n[15,13]=<p099>\n[15,15

In [57]:

base = "Qwen/Qwen3-0.6B"
ckpt = "runs/lora_bond_qwen/checkpoint-300"

tokenizer = AutoTokenizer.from_pretrained(base, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(base, device_map="auto", trust_remote_code=True)
model = PeftModel.from_pretrained(model, ckpt)

model.eval()


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 1024)
        (layers): ModuleList(
          (0-27): 28 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1024, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [25]:
prompt

'<|im_start|>system\nYou are a chemist. For each atom pair within 2 Å classify the bond type. Labels: 0 no-bond, 1 single, 2 double, 3 triple, 4 aromatic. Return ONLY JSON list [{"pair":[i,j],"label":n}].<|im_end|>\n<|im_start|>user\nPairs within 2 Å:\n[0,0]=<p077>\n[0,1]=<p121>\n[1,0]=<p046>\n[1,1]=<p117>\n[1,2]=<p115>\n[2,1]=<p115>\n[2,2]=<p060>\n[2,3]=<p113>\n[3,2]=<p115>\n[3,3]=<p025>\n[3,4]=<p124>\n[3,5]=<p083>\n[4,3]=<p010>\n[4,4]=<p117>\n[5,3]=<p083>\n[5,5]=<p025>\n[5,6]=<p107>\n[5,27]=<p065>\n[6,5]=<p028>\n[6,6]=<p117>\n[6,7]=<p115>\n[7,6]=<p115>\n[7,7]=<p060>\n[7,8]=<p115>\n[8,7]=<p115>\n[8,8]=<p117>\n[8,9]=<p016>\n[8,26]=<p086>\n[9,8]=<p038>\n[9,9]=<p034>\n[9,10]=<p017>\n[9,11]=<p096>\n[10,9]=<p016>\n[10,10]=<p117>\n[11,9]=<p032>\n[11,11]=<p025>\n[11,12]=<p120>\n[12,11]=<p120>\n[12,12]=<p060>\n[12,13]=<p016>\n[12,25]=<p104>\n[13,12]=<p038>\n[13,13]=<p034>\n[13,14]=<p075>\n[13,15]=<p096>\n[13,16]=<p019>\n[14,13]=<p096>\n[14,14]=<p025>\n[15,13]=<p099>\n[15,15]=<p025>\n[16,13]=<

In [51]:
input_ids

tensor([[151644,   8948,    198,  ...,  55992, 151645,    198]],
       device='cuda:1')

In [49]:
input_ids = tokenizer.apply_chat_template(
    messages_no_answer,
    return_tensors="pt",
    add_special_tokens=False  # важно оставить False для совместимости
).to(model.device)

In [59]:
input_ids = tokenizer.apply_chat_template(
    messages_no_answer,
    return_tensors="pt",
    add_special_tokens=False
).to(model.device)

with torch.no_grad():
    output = model.generate(
        input_ids=input_ids,
        max_new_tokens=1024,  # ограничим генерацию разумным числом
        do_sample=False,
        eos_token_id=tokenizer.convert_tokens_to_ids("<|im_end|>"),
        pad_token_id=tokenizer.pad_token_id,
    )

generated_text = tokenizer.decode(
    output[0][input_ids.shape[-1]:],
    skip_special_tokens=True
).strip()

print(generated_text)

In [3]:
generated_text

NameError: name 'generated_text' is not defined

In [1]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("AI4Chem/ChemLLM-7B-Chat-1_5-DPO", trust_remote_code=True)

for name, module in model.named_modules():
    if hasattr(module, "weight") or hasattr(module, "in_features"):
        print(name)


/home/user12/.conda/envs/pasha/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 8/8 [00:02<00:00,  2.75it/s]


model.tok_embeddings
model.layers.0.attention.wqkv
model.layers.0.attention.wo
model.layers.0.feed_forward.w1
model.layers.0.feed_forward.w3
model.layers.0.feed_forward.w2
model.layers.0.attention_norm
model.layers.0.ffn_norm
model.layers.1.attention.wqkv
model.layers.1.attention.wo
model.layers.1.feed_forward.w1
model.layers.1.feed_forward.w3
model.layers.1.feed_forward.w2
model.layers.1.attention_norm
model.layers.1.ffn_norm
model.layers.2.attention.wqkv
model.layers.2.attention.wo
model.layers.2.feed_forward.w1
model.layers.2.feed_forward.w3
model.layers.2.feed_forward.w2
model.layers.2.attention_norm
model.layers.2.ffn_norm
model.layers.3.attention.wqkv
model.layers.3.attention.wo
model.layers.3.feed_forward.w1
model.layers.3.feed_forward.w3
model.layers.3.feed_forward.w2
model.layers.3.attention_norm
model.layers.3.ffn_norm
model.layers.4.attention.wqkv
model.layers.4.attention.wo
model.layers.4.feed_forward.w1
model.layers.4.feed_forward.w3
model.layers.4.feed_forward.w2
model.la

In [38]:
# print(tokenizer.convert_tokens_to_ids("<|im_end|>")) 
# print(tokenizer.eos_token, tokenizer.eos_token_id)
print(tokenizer.decode(tokenizer(prompt)["input_ids"]))

<|im_start|>system
You are a chemist. For each atom pair within 2 Å classify the bond type. Labels: 0 no-bond, 1 single, 2 double, 3 triple, 4 aromatic. Return ONLY JSON list [{"pair":[i,j],"label":n}].<|im_end|>
<|im_start|>user
Pairs within 2 Å:
[0,0]=<p077>
[0,1]=<p121>
[1,0]=<p046>
[1,1]=<p117>
[1,2]=<p115>
[2,1]=<p115>
[2,2]=<p060>
[2,3]=<p113>
[3,2]=<p115>
[3,3]=<p025>
[3,4]=<p124>
[3,5]=<p083>
[4,3]=<p010>
[4,4]=<p117>
[5,3]=<p083>
[5,5]=<p025>
[5,6]=<p107>
[5,27]=<p065>
[6,5]=<p028>
[6,6]=<p117>
[6,7]=<p115>
[7,6]=<p115>
[7,7]=<p060>
[7,8]=<p115>
[8,7]=<p115>
[8,8]=<p117>
[8,9]=<p016>
[8,26]=<p086>
[9,8]=<p038>
[9,9]=<p034>
[9,10]=<p017>
[9,11]=<p096>
[10,9]=<p016>
[10,10]=<p117>
[11,9]=<p032>
[11,11]=<p025>
[11,12]=<p120>
[12,11]=<p120>
[12,12]=<p060>
[12,13]=<p016>
[12,25]=<p104>
[13,12]=<p038>
[13,13]=<p034>
[13,14]=<p075>
[13,15]=<p096>
[13,16]=<p019>
[14,13]=<p096>
[14,14]=<p025>
[15,13]=<p099>
[15,15]=<p025>
[16,13]=<p099>
[16,16]=<p025>
[16,17]=<p058>
[16,21]=<p127>
[17,

In [37]:
'im' in generated


False